# Capstone Project: Data Collection
* Collecting Insider, House of Reps, Senate, and Congress data using Quiver quant alternative data source
* quiverquant rate limit: 100calls/second and 1000 calls per minute
* for each data set pull all tickers in SP500 and all recent trade tickers (if not in SP500)

In [12]:
! pip install jsonmerge

  Using cached jsonmerge-1.9.0.tar.gz (32 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for jsonmerge: filename=jsonmerge-1.9.0-py3-none-any.whl size=18609 sha256=808b8ed62f0c0b73e2d22151bd955df41bb84b62f07b7bd3391f8abefb1f7a86
  Stored in directory: /Users/austinpaxton/Library/Caches/pip/wheels/45/30/22/18d39219b08402f26539c3e72a132353a31f49204ff35c8d8e
Successfully built jsonmerge


In [41]:
import json
from json import JSONDecodeError
import requests
from pandas import json_normalize
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import math


f = open("/Users/austinpaxton/Documents/credentials/quiver_quant_cred.json")
data = json.load(f)

auth_token = data["auth_token"]
base_url = data["base_URL"]
print(base_url)

headers = {
    'accept': 'application/json',
    'X-CSRFToken': 'TyTJwjuEC7VV7mOqZ622haRaaUr0x0Ng4nrwSRFKQs7vdoBcJlK9qjAS69ghzhFu',
    'Authorization': f"Token {auth_token}"
}


https://api.quiverquant.com/beta/


In [42]:
#pull SP500 tickers
sp500_tickers = pd.read_csv("SPY_companies.csv")
sp500_tickers = sp500_tickers.Symbol.tolist()
print(f"Length: {len(sp500_tickers)}")
#print(sp500_tickers)

Length: 502


In [43]:
# pull larger list of all tickers (5783 total) for more data
tickers_all = pd.read_csv("stock_info.csv")
tickers_all = tickers_all.Ticker.unique().tolist()
print(len(tickers_all))
tickers_all = list(set(tickers_all+sp500_tickers))
print(len(tickers_all))


5779
5867


In [38]:
def pull_hist_data(data,tickers):
    # iteratively pull government contracts
    from jsonmerge import merge
    import time

    #initialize variables
    request_qty = 1000  #need to keep track of requests... rate limited to 1000 requests per minute
    empty_response_tickers = []
    data_merged= []
    first_response = True

    #gets time of first request so we know when to refresh the request qty
    rate_lim_reset_time = datetime.now() + timedelta(minutes=1)  
    
    for ticker in tickers:
        time_till_refresh = rate_lim_reset_time-datetime.now()
        wait_time = math.ceil(time_till_refresh.total_seconds())
        print(f"Rate Limit Reset Time: {rate_lim_reset_time} Wait Time: {wait_time}s || Request Qty: {request_qty}")
        if wait_time<=0:
            print("Wait Time<=0")
            request_qty = 1000
            print(f"Wait Time<=0 || request qty: {request_qty}")
            rate_lim_reset_time = datetime.now() + timedelta(minutes=1)
            
        elif request_qty<=0:  
            print(f"No requests left || Request Qty: {request_qty}")
            time_till_refresh = rate_lim_reset_time-datetime.now()
            print(f"Time untill refresh: {time_till_refresh}")
            wait_time = math.ceil(time_till_refresh.total_seconds())
            print(f"Wait Time: {wait_time}")
            time.sleep(wait_time)
            
            request_qty = 1000
            print(f"Requests Refreshed || Request Qty: {request_qty}")
            rate_lim_reset_time = datetime.now() + timedelta(minutes=1) #refresh reset time
            print(f"Reset Time Refreshed || Rate Limit Reset Time {rate_lim_reset_time}")
            
        else:
            try:
                url = base_url + f"historical/{data}/{ticker}"
                data_temp = requests.get(url, headers=headers, timeout=40).json()
                print(f"\n--------Pulling {data} on Ticker: {ticker}  || {1+tickers.index(ticker)}/{len(tickers)}------")
                request_qty -= 1

                data_temp = json_normalize(data_temp) #convert data to json

                empty = data_temp.empty  #check if data is empty

                #stores data if not empty and merges to existing pandas df
                if not empty:
                    print("Status: Recieved Data")
                    if first_response:
                        data_merged = data_temp
                        first_response = False
                    else:
                        try:
                            data_merged = data_merged.merge(data_temp,how="outer")
                        except: 
                            print("!!!!!!!!!!!!!!!merge failed!!!!!!!!!!!!!!!!!!!!")
                            print(data_temp)
                            break

                else:
                    empty_response_tickers.append(ticker)
                    print("Status: Empty Response")
            except JSONDecodeError: 
                print(f"!!!!!!!!!!!!!!!! JSON Decode Error on {ticker}!!!!!!!!!!!!!!!!!!!!!!")

        
    return data_merged, empty_response_tickers




In [40]:
contract_data, empty_response = pull_hist_data(data="govcontractsall",tickers=tickers_all)

Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 60s || Request Qty: 1000

--------Pulling govcontractsall on Ticker: CYBX  || 1/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 60s || Request Qty: 999

--------Pulling govcontractsall on Ticker: STX  || 2/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 60s || Request Qty: 998

--------Pulling govcontractsall on Ticker: MAR  || 3/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 60s || Request Qty: 997

--------Pulling govcontractsall on Ticker: IAE  || 4/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 60s || Request Qty: 996

--------Pulling govcontractsall on Ticker: OME  || 5/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 59s || Request Qty: 995

--------Pulling govcontractsall on Ticker: IMCB


--------Pulling govcontractsall on Ticker: GPM  || 49/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 48s || Request Qty: 951

--------Pulling govcontractsall on Ticker: FBHS  || 50/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 48s || Request Qty: 950

--------Pulling govcontractsall on Ticker: CCV  || 51/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 47s || Request Qty: 949

--------Pulling govcontractsall on Ticker: IMOS  || 52/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 47s || Request Qty: 948

--------Pulling govcontractsall on Ticker: SGZA  || 53/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 47s || Request Qty: 947

--------Pulling govcontractsall on Ticker: BKMU  || 54/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:


--------Pulling govcontractsall on Ticker: TEO  || 96/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 37s || Request Qty: 904

--------Pulling govcontractsall on Ticker: WLB  || 97/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 37s || Request Qty: 903

--------Pulling govcontractsall on Ticker: NVX-C  || 98/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 36s || Request Qty: 902

--------Pulling govcontractsall on Ticker: GJT  || 99/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 36s || Request Qty: 901

--------Pulling govcontractsall on Ticker: PTSI  || 100/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 35s || Request Qty: 900

--------Pulling govcontractsall on Ticker: KMI  || 101/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23


--------Pulling govcontractsall on Ticker: BSRR  || 144/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 23s || Request Qty: 856

--------Pulling govcontractsall on Ticker: SANW  || 145/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 23s || Request Qty: 855

--------Pulling govcontractsall on Ticker: BKU  || 146/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 22s || Request Qty: 854

--------Pulling govcontractsall on Ticker: IRL  || 147/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 22s || Request Qty: 853

--------Pulling govcontractsall on Ticker: AEF  || 148/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 22s || Request Qty: 852

--------Pulling govcontractsall on Ticker: NXK-C  || 149/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-


--------Pulling govcontractsall on Ticker: VIA  || 191/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 13s || Request Qty: 809

--------Pulling govcontractsall on Ticker: SLVO  || 192/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 12s || Request Qty: 808

--------Pulling govcontractsall on Ticker: FWLT  || 193/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 12s || Request Qty: 807

--------Pulling govcontractsall on Ticker: TPL  || 194/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 12s || Request Qty: 806

--------Pulling govcontractsall on Ticker: MSEX  || 195/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:31:18.625354 Wait Time: 12s || Request Qty: 805

--------Pulling govcontractsall on Ticker: NEE-F  || 196/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04

...


--------Pulling govcontractsall on Ticker: PVA  || 5633/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 47s || Request Qty: 962

--------Pulling govcontractsall on Ticker: RPRX  || 5634/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 47s || Request Qty: 961

--------Pulling govcontractsall on Ticker: COR-A  || 5635/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 47s || Request Qty: 960

--------Pulling govcontractsall on Ticker: UCP  || 5636/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 47s || Request Qty: 959

--------Pulling govcontractsall on Ticker: C  || 5637/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 46s || Request Qty: 958

--------Pulling govcontractsall on Ticker: IXYS  || 5638/5867------
Status: Empty Response
Rate Limit Reset Time: 2023


--------Pulling govcontractsall on Ticker: MGU  || 5681/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 32s || Request Qty: 914

--------Pulling govcontractsall on Ticker: NLY  || 5682/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 32s || Request Qty: 913

--------Pulling govcontractsall on Ticker: PCRX  || 5683/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 31s || Request Qty: 912

--------Pulling govcontractsall on Ticker: BCC  || 5684/5867------
Status: Recieved Data
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 29s || Request Qty: 911

--------Pulling govcontractsall on Ticker: PNM  || 5685/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 29s || Request Qty: 910

--------Pulling govcontractsall on Ticker: JACQW  || 5686/5867------
Status: Empty Response
Rate Limit Reset Time: 2023


--------Pulling govcontractsall on Ticker: SSP  || 5728/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 11s || Request Qty: 867

--------Pulling govcontractsall on Ticker: NCU-C  || 5729/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 11s || Request Qty: 866

--------Pulling govcontractsall on Ticker: CWCO  || 5730/5867------
Status: Recieved Data
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 9s || Request Qty: 865

--------Pulling govcontractsall on Ticker: MBT  || 5731/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 9s || Request Qty: 864

--------Pulling govcontractsall on Ticker: NTE  || 5732/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-09 23:59:06.670805 Wait Time: 8s || Request Qty: 863

--------Pulling govcontractsall on Ticker: BXS-A  || 5733/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-


--------Pulling govcontractsall on Ticker: DBL  || 5776/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-10 00:00:06.775923 Wait Time: 46s || Request Qty: 973

--------Pulling govcontractsall on Ticker: JZJ  || 5777/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-10 00:00:06.775923 Wait Time: 46s || Request Qty: 972

--------Pulling govcontractsall on Ticker: SMSI  || 5778/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-10 00:00:06.775923 Wait Time: 45s || Request Qty: 971

--------Pulling govcontractsall on Ticker: MELI  || 5779/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-10 00:00:06.775923 Wait Time: 45s || Request Qty: 970

--------Pulling govcontractsall on Ticker: SVNT  || 5780/5867------
Status: Empty Response
Rate Limit Reset Time: 2023-04-10 00:00:06.775923 Wait Time: 45s || Request Qty: 969

--------Pulling govcontractsall on Ticker: MSF  || 5781/5867------
Status: Empty Response
Rate Limit Reset Time: 202

ReadTimeout: HTTPSConnectionPool(host='api.quiverquant.com', port=443): Read timed out. (read timeout=40)

In [16]:
lobbying_data = pull_hist_data(data="lobbying", tickers=tickers_all)


--------Pulling lobbying on Ticker: CYBX  || 1/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: STX  || 2/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MAR  || 3/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: IAE  || 4/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: OME  || 5/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: IMCB  || 6/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: TRC  || 7/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: ILMN  || 8/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: CAJ  || 9/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: AMT  || 10/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: LXK  || 11/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: NHTB  || 12/5867------
Status: Empty Response

--------Pulling lobbying

Status: Recieved Data

--------Pulling lobbying on Ticker: SYNA  || 102/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: XOM  || 103/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: ALLB  || 104/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: ULTA  || 105/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: BTO  || 106/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: TCCA  || 107/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: FDI  || 108/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: FIX  || 109/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: VLTC  || 110/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: TKMR  || 111/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: PSA-R  || 112/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: HYB  || 113/5867------


--------Pulling lobbying on Ticker: ENTG  || 200/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: SGNT  || 201/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MSLI  || 202/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: CBB-B  || 203/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: EPIQ  || 204/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: SEAS  || 205/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: RCPT  || 206/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: SPF  || 207/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MOBI  || 208/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: FCTY  || 209/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: UBSH  || 210/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: CENX  || 211/5867------
Status: Recieved


--------Pulling lobbying on Ticker: LYB  || 299/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: NRG  || 300/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: DAKT  || 301/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MPW  || 302/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: CBG  || 303/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: ARE  || 304/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: SPH  || 305/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: ANCX  || 306/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: GTIV  || 307/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: EXAS  || 308/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: SIR  || 309/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: VRTB  || 310/5867------
Status: Empty Response




--------Pulling lobbying on Ticker: FLDM  || 398/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: CALX  || 399/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: MBFI  || 400/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: HOG  || 401/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: PSAU  || 402/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: APAM  || 403/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: OXLC  || 404/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: HEOP  || 405/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MYE  || 406/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: TFX  || 407/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: PVTBP  || 408/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: GTWN  || 409/5867------
Status: Empty Respons

...


--------Pulling lobbying on Ticker: GLAD  || 895/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: OXLCP  || 896/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MYC  || 897/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: ORB  || 898/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: G  || 899/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: TZOO  || 900/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: CBD  || 901/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: UGI  || 902/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: PSCF  || 903/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: CVBF  || 904/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: LXP  || 905/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: LVB  || 906/5867------
Status: Empty Response



--------Pulling lobbying on Ticker: NSS  || 994/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: END  || 995/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: USMO  || 996/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: BGG  || 997/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: GCBC  || 998/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: GLBS  || 999/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: ONEQ  || 1000/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MUR  || 1002/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: BHLB  || 1003/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MTX  || 1004/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: ZINC  || 1005/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MHNB  || 1006/5867------
Status: Empty Re


--------Pulling lobbying on Ticker: KST  || 1093/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: GGT  || 1094/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: ODFL  || 1095/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: SEAC  || 1096/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: BKI  || 1097/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: MSO  || 1098/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: PRI  || 1099/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: EOD  || 1100/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: SUBK  || 1101/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: UTI  || 1102/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: MHN  || 1103/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: PJC  || 1104/5867------
Status: Empty R


--------Pulling lobbying on Ticker: MNK  || 1191/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: IRE-B  || 1192/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: ROCM  || 1193/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: AVA  || 1194/5867------
Status: Recieved Data

--------Pulling lobbying on Ticker: TSN  || 1195/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: LLY  || 1196/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: STP  || 1197/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: VNRAP  || 1198/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: CBST  || 1199/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: TCRD  || 1200/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: TSRO  || 1201/5867------
Status: Empty Response

--------Pulling lobbying on Ticker: TRGP  || 1202/5867------
Status:


--------Pulling lobbying on Ticker: PFD  || 1290/5867------
Status: Recieved Data
!!!!!!!!!!!!!!!merge failed!!!!!!!!!!!!!!!!!!!!
                                              detail
0  Request was throttled. Expected available in 2...


In [6]:
contract_data.to_csv("/Users/austinpaxton/Documents/ML_Bootcamp/Capstone/contract_data_04-09-23.csv")

NameError: name 'contract_data' is not defined